# Clustering PGCLCs

### Resources
* Tutorial scanpy https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html
* Tutorial scvi https://docs.scvi-tools.org/en/stable/tutorials/notebooks/scrna/harmonization.html

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import scvi
import seaborn as sns
import torch

In [2]:
print(sc.__version__)
print(torch.__version__)
print(pd.__version__)
print(scvi.__version__)
print(scvi)
print(torch.cuda.is_available())

1.9.8
2.2.1
2.2.1
1.1.2
<module 'scvi' from '/data1/imoustakas/miniforge3/envs/scvi/lib/python3.9/site-packages/scvi/__init__.py'>
False


In [5]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

scanpy==1.9.8 anndata==0.10.6 umap==0.5.5 numpy==1.26.4 scipy==1.12.0 pandas==2.2.1 scikit-learn==1.4.1.post1 statsmodels==0.14.1 pynndescent==0.5.11


## Load h5ad, output of Seura tworkflow

In [4]:
CR27_d15PGCLC = sc.read_10x_mtx("/data1/imoustakas/project-477-PGCLC_10x_yolanda/PGCLC_counts/CR27_d15PGCLC")
CR27_d5PGCLC = sc.read_10x_mtx("/data1/imoustakas/project-477-PGCLC_10x_yolanda/PGCLC_counts/CR27_d5PGCLC")
MB12_d5PGCLC = sc.read_10x_mtx("/data1/imoustakas/project-477-PGCLC_10x_yolanda/PGCLC_counts/MB12_d5PGCLC")
YD90_d15PGCLC = sc.read_10x_mtx("/data1/imoustakas/project-477-PGCLC_10x_yolanda/PGCLC_counts/YD90_d15PGCLC")

In [28]:
adata = sc.concat([CR27_d15PGCLC, CR27_d5PGCLC, MB12_d5PGCLC, YD90_d15PGCLC], 
                  label="run",
                  keys=["CR27_d15", "CR27_d5", "MB12_d5", "YD90_d15"],
                  index_unique="_")
# adata.obs_names_make_unique()
adata

AnnData object with n_obs × n_vars = 32537 × 36604
    obs: 'run'

In [29]:
adata.obs["run"]

AAACCCAAGAGCAAGA_CR27_d15    CR27_d15
AAACCCAAGCAGTACG_CR27_d15    CR27_d15
AAACCCAAGGCAGGGA_CR27_d15    CR27_d15
AAACCCACACGGCGTT_CR27_d15    CR27_d15
AAACCCACAGCACAAG_CR27_d15    CR27_d15
                               ...   
TTTGTTGGTTGGATCT_YD90_d15    YD90_d15
TTTGTTGTCACCTCAC_YD90_d15    YD90_d15
TTTGTTGTCATTCATC_YD90_d15    YD90_d15
TTTGTTGTCCACGTAA_YD90_d15    YD90_d15
TTTGTTGTCCCGAGGT_YD90_d15    YD90_d15
Name: run, Length: 32537, dtype: category
Categories (4, object): ['CR27_d15', 'CR27_d5', 'MB12_d5', 'YD90_d15']

## Basic Filtering

In [13]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
adata

AnnData object with n_obs × n_vars = 32537 × 28557
    obs: 'n_genes'
    var: 'n_cells'

In [12]:
adata.X.toarray()[1:40, 1:20]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 3.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

## Load the Seurat object and extract the cells IDs
These cells are past filtering (usual QC and doublet removal)

In [14]:
adata_seurat = sc.read_h5ad("PGCLCs.h5ad")

/data1/imoustakas/miniforge3/envs/scvi/lib/python3.9/site-packages/anndata/compat/__init__.py:311: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(


In [46]:
result = adata_seurat.obs_names.str.split(pat=r'_(?!.*_)')
adata_seurat.obs_names = [l[1] + "_" + l[0] for l in result]
adata_seurat.obs_names


Index(['AAACCCACAAACCGGA_CR27_d5', 'AAACCCACACTTGTGA_CR27_d5',
       'AAACCCACAGTCTTCC_CR27_d5', 'AAACCCAGTCTAATCG_CR27_d5',
       'AAACCCATCATAGGCT_CR27_d5', 'AAACGAACAACCAATC_CR27_d5',
       'AAACGAACAATTCTCT_CR27_d5', 'AAACGAACACCAGTAT_CR27_d5',
       'AAACGAACAGTCTACA_CR27_d5', 'AAACGCTAGTCTCCTC_CR27_d5',
       ...
       'TTTGGTTTCCTGTTAT_YD90_d15', 'TTTGGTTTCTCGCGTT_YD90_d15',
       'TTTGTTGCACAAATGA_YD90_d15', 'TTTGTTGCAGCGGATA_YD90_d15',
       'TTTGTTGCATCTAGAC_YD90_d15', 'TTTGTTGGTTGGATCT_YD90_d15',
       'TTTGTTGTCACCTCAC_YD90_d15', 'TTTGTTGTCATTCATC_YD90_d15',
       'TTTGTTGTCCACGTAA_YD90_d15', 'TTTGTTGTCCCGAGGT_YD90_d15'],
      dtype='object', length=26383)

In [52]:
adata = adata[adata.obs_names.isin(adata_seurat.obs_names), :]

In [56]:
adata.X

<26383x36604 sparse matrix of type '<class 'numpy.float32'>'
	with 74956827 stored elements in Compressed Sparse Row format>

In [9]:
adata.X = adata.raw.X
adata.X.toarray()[1:20, 1:20]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.   

## Basic Filtering

In [11]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

filtered out 8647 genes that are detected in less than 3 cells


In [12]:
adata

AnnData object with n_obs × n_vars = 26383 × 27957
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'run', 'percent.mt', 'percent.dissoc', 'dbl_finder_score', 'dbl_finder_class', 'RNA_snn_res.0.5', 'seurat_clusters', 'n_genes'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable', 'n_cells'
    uns: 'neighbors'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'distances'

In [13]:
adata.X.toarray()[1:20, 1:20]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 1.91009642, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 2.27668416, 2.27668416, 0.        , 0.        ,
        0.   

Citing from “Simple Single Cell” workflows (Lun, McCarthy & Marioni, 2017):

`High proportions are indicative of poor-quality cells (Islam et al. 2014; Ilicic et al. 2016), possibly because of loss of cytoplasmic RNA from perforated cells. The reasoning is that mitochondria are larger than individual transcript molecules and less likely to escape through tears in the cell membrane.`


In [ ]:
# annotate the group of mitochondrial genes as "mt"
adata.var["mt"] = adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True
)

: 

## Viollin plots
* Note that using the default violin plot function, where the plots are placed side by side, failsl with an error msg

In [ ]:
# sc.pl.violin(
#     adata,
#     ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
#     jitter=0.4,
#     multi_panel=True,
# )

sc.pl.violin(
    adata,
    ["n_genes_by_counts"],
    jitter=0.4,
    multi_panel=False,
)
sc.pl.violin(
    adata,
    ["total_counts"],
    jitter=0.4,
    multi_panel=False,
)
sc.pl.violin(
    adata,
    ["pct_counts_mt"],
    jitter=0.4,
    multi_panel=False,
)

: 

In [ ]:
sc.pl.scatter(adata, x="total_counts", y="pct_counts_mt")
sc.pl.scatter(adata, x="total_counts", y="n_genes_by_counts")

: 

In [ ]:
adata = adata[adata.obs.n_genes_by_counts < 9000, :]
adata = adata[adata.obs.pct_counts_mt < 12, :].copy()

: 

In [ ]:
adata

: 

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts"],
    jitter=0.4,
    multi_panel=False,
)
sc.pl.violin(
    adata,
    ["total_counts"],
    jitter=0.4,
    multi_panel=False,
)
sc.pl.violin(
    adata,
    ["pct_counts_mt"],
    jitter=0.4,
    multi_panel=False,
)

: 

In [ ]:
adata.raw = adata

: 

In [ ]:
# sc.pp.normalize_total(adata, target_sum=5e4)
# sc.pp.log1p(adata)

: 

## Select the top 2000 most variable genes

In [ ]:
sc.pp.highly_variable_genes(
    adata,
    flavor="seurat_v3",
    n_top_genes=2000,
    subset=True,
)

: 

In [ ]:
adata

: 

In [ ]:
scvi.model.SCVI.setup_anndata(adata)

: 

In [ ]:
model = scvi.model.SCVI(adata)

: 

In [ ]:
model.train()

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

In [ ]:
sc.pp.normalize_total(adata, target_sum=5e4)

: 

In [ ]:
sc.pp.log1p(adata)

: 

In [ ]:
sc.pp.highly_variable_genes(adata, 
                            n_top_genes=2000)

: 

In [ ]:
sc.pl.highly_variable_genes(adata)

: 

In [ ]:
adata

: 

In [ ]:
adata.X.toarray()

: 

In [ ]:
adata.raw[:, adata.var.highly_variable].shape

: 

In [ ]:
test = adata.raw[:, adata.var.highly_variable]

: 

In [ ]:
test.

: 

: 

: 